Model training

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import shap
import joblib
import matplotlib.pyplot as plt
import random
import matplotlib.ticker as ticker
from matplotlib.ticker import MaxNLocator

random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

data = pd.read_excel(r'Original_data')

feature_names = data.columns[2:17]

# data standardization
scaler = MinMaxScaler(feature_range=(-1, 1))
data.loc[:, data.columns[2:17]] = scaler.fit_transform(data.loc[:, data.columns[2:17]])
scaler_filename = 'scaler.pkl'
joblib.dump(scaler, scaler_filename)

# input and output
X = data.iloc[:, 2:17].values
y = data.iloc[:, 17].values

# data splitting based on Isotherm
unique_isotherms = data['Isotherm'].unique()
train_indices = []
test_indices = []

num_isotherms_test = int(0.1 * len(unique_isotherms))  # 10% for test-set

test_isotherms = np.random.choice(unique_isotherms, size=num_isotherms_test, replace=False)
remaining_isotherms = np.setdiff1d(unique_isotherms, test_isotherms)

for isotherm in unique_isotherms:
    indices = data[data['Isotherm'] == isotherm].index

    if isotherm in test_isotherms:
        test_indices.extend(indices)
    else:
        train_indices.extend(indices)

X_test = X[test_indices]
y_test = y[test_indices]
X_train = X[train_indices]
y_train = y[train_indices]

# model construction
model = XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, objective='reg:squarederror')

kf = KFold(n_splits=5, shuffle=True)
y_pred = cross_val_predict(model, X_train, y_train, cv=kf)

# model evaluation
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

print("XGBoost training results:")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R2 Score: {r2:.4f}")

model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("XGBoost testing results:")
print(f"RMSE: {rmse_test:.4f}")
print(f"MAE: {mae_test:.4f}")
print(f"R2 Score: {r2_test:.4f}")

# Save the model
model_filename = 'PWM_XGBoost.joblib'
joblib.dump(model, model_filename)


SHAP analysis based on OMPs classification into high/low hydrophobic

In [ ]:
data_original = pd.read_excel(r'Original_data')

# Classification based on whether or not the logD (pH=7.4) is > 0
highlogD_indices = data_original[data_original.iloc[:, 2] > 2].index
lowlogD_indices = data_original[data_original.iloc[:, 2] <= 2].index

# local SHAP calculation
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

shap_highlogD_df = pd.DataFrame(columns=['Isotherm', *feature_names])
shap_lowlogD_df = pd.DataFrame(columns=['Isotherm', *feature_names])

for isotherm in sorted(data['Isotherm'].unique()):
    iso_indices = data[data['Isotherm'] == isotherm].index
    shap_values_iso = shap_values[iso_indices]
    
    if any(idx in highlogD_indices for idx in iso_indices):
        shap_df_iso = pd.DataFrame(shap_values_iso, columns=feature_names)
        shap_df_iso['Isotherm'] = isotherm
        shap_highlogD_df = pd.concat([shap_highlogD_df, shap_df_iso], ignore_index=True)
    elif any(idx in lowlogD_indices for idx in iso_indices):
        shap_df_iso = pd.DataFrame(shap_values_iso, columns=feature_names)
        shap_df_iso['Isotherm'] = isotherm
        shap_lowlogD_df = pd.concat([shap_lowlogD_df, shap_df_iso], ignore_index=True)

shap_highlogD_df = shap_highlogD_df.sort_values(by='Isotherm')
shap_lowlogD_df = shap_lowlogD_df.sort_values(by='Isotherm')



In [ ]:
target_feature_name = feature_names[0]
target_feature_index = feature_names.get_loc(target_feature_name)

# LogD extraction
highlogD_feature_values = data_original.iloc[highlogD_indices, target_feature_index + 2]
lowlogD_feature_values = data_original.iloc[lowlogD_indices, target_feature_index + 2]

# SHAP extraction
highlogD_shap_values = shap_highlogD_df.iloc[:, target_feature_index + 1]
lowlogD_shap_values = shap_lowlogD_df.iloc[:, target_feature_index + 1]

# Trendline
highlogD_trend_line = np.polyfit(highlogD_feature_values, highlogD_shap_values, 1)
highlogD_trend_line_y = np.polyval(highlogD_trend_line, highlogD_feature_values)
highlogD_r2 = r2_score(highlogD_shap_values, highlogD_trend_line_y)

lowlogD_trend_line = np.polyfit(lowlogD_feature_values, lowlogD_shap_values, 1)
lowlogD_trend_line_y = np.polyval(lowlogD_trend_line, lowlogD_feature_values)
lowlogD_r2 = r2_score(lowlogD_shap_values, lowlogD_trend_line_y)

highlogD_slope = highlogD_trend_line[0]
lowlogD_slope = lowlogD_trend_line[0]

# Local SHAP plotting
plt.figure(figsize=(20, 8))

plt.plot(lowlogD_feature_values, lowlogD_shap_values, 'o', color=(17/255, 69/255, 145/255), label=f'logD <=2 (Slope={highlogD_slope:.2f})', alpha=0.5, markersize=20)
plt.plot(highlogD_feature_values, highlogD_shap_values, 'o', color=(17/255, 69/255, 145/255), label=f'logD >2 (Slope={lowlogD_slope:.2f})', alpha=0.5, markersize=20)

plt.plot(lowlogD_feature_values, lowlogD_trend_line_y, alpha=1, color=(17/255, 69/255, 145/255), linewidth=7)
plt.plot(highlogD_feature_values, highlogD_trend_line_y, alpha=1, color=(17/255, 69/255, 145/255), linewidth=7)

plt.xlabel(target_feature_name, fontsize=40, labelpad=25)
plt.ylabel('SHAP Value', fontsize=40, labelpad=25)

ax = plt.gca()
ax.spines['top'].set_linewidth(3)
ax.spines['right'].set_linewidth(3)
ax.spines['left'].set_linewidth(3)
ax.spines['bottom'].set_linewidth(3)

ax.tick_params(axis='both', which='major', width=3, length=10)

plt.gca().xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))

ax.yaxis.set_major_locator(MaxNLocator(nbins=6))

plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.legend(fontsize=40, frameon=False)

plt.show()


SHAP analysis based on OMPs classification into aromatic/aliphatic

In [ ]:
data_original = pd.read_excel(r'Original_data')

# Classification based on whether or not the number of aromatic rings is 0
aromatic_indices = data_original[data_original.iloc[:, 1] != 0].index
non_aromatic_indices = data_original[data_original.iloc[:, 1] == 0].index

# local SHAP calculation
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

shap_aromatic_df = pd.DataFrame(columns=['Isotherm', *feature_names])
shap_non_aromatic_df = pd.DataFrame(columns=['Isotherm', *feature_names])

for isotherm in sorted(data['Isotherm'].unique()):
    iso_indices = data[data['Isotherm'] == isotherm].index
    shap_values_iso = shap_values[iso_indices]
    
    if any(idx in aromatic_indices for idx in iso_indices):
        shap_df_iso = pd.DataFrame(shap_values_iso, columns=feature_names)
        shap_df_iso['Isotherm'] = isotherm
        shap_aromatic_df = pd.concat([shap_aromatic_df, shap_df_iso], ignore_index=True)
    elif any(idx in non_aromatic_indices for idx in iso_indices):
        shap_df_iso = pd.DataFrame(shap_values_iso, columns=feature_names)
        shap_df_iso['Isotherm'] = isotherm
        shap_non_aromatic_df = pd.concat([shap_non_aromatic_df, shap_df_iso], ignore_index=True)

shap_aromatic_df = shap_aromatic_df.sort_values(by='Isotherm')
shap_non_aromatic_df = shap_non_aromatic_df.sort_values(by='Isotherm')


In [ ]:
target_feature_name = feature_names[0]
target_feature_index = feature_names.get_loc(target_feature_name)

# LogD extraction
aromatic_feature_values = data_original.iloc[aromatic_indices, target_feature_index + 2].reset_index(drop=True)
non_aromatic_feature_values = data_original.iloc[non_aromatic_indices, target_feature_index + 2].reset_index(drop=True)

# SHAP extraction
aromatic_shap_values = shap_aromatic_df.iloc[:, target_feature_index + 1].reset_index(drop=True)
non_aromatic_shap_values = shap_non_aromatic_df.iloc[:, target_feature_index + 1].reset_index(drop=True)

# Trendline
aromatic_trend_line = np.polyfit(aromatic_feature_values, aromatic_shap_values, 1)
aromatic_trend_line_y = np.polyval(aromatic_trend_line, aromatic_feature_values)
aromatic_r2 = r2_score(aromatic_shap_values, aromatic_trend_line_y)

non_aromatic_trend_line = np.polyfit(non_aromatic_feature_values, non_aromatic_shap_values, 1)
non_aromatic_trend_line_y = np.polyval(non_aromatic_trend_line, non_aromatic_feature_values)
non_aromatic_r2 = r2_score(non_aromatic_shap_values, non_aromatic_trend_line_y)

aromatic_slope = aromatic_trend_line[0]
non_aromatic_slope = non_aromatic_trend_line[0]

# Local SHAP plotting
plt.rcParams['font.family'] = 'Arial'

plt.figure(figsize=(20, 8))

plt.plot(non_aromatic_feature_values, non_aromatic_shap_values, 'o', color='orange', label=f'Aliphatic (Slope={non_aromatic_slope:.2f})', alpha=0.25, markersize=20)
plt.plot(aromatic_feature_values, aromatic_shap_values, 'o', color='green', label=f'Aromatic (Slope={aromatic_slope:.2f})', alpha=0.25, markersize=20)

plt.plot(non_aromatic_feature_values, non_aromatic_trend_line_y, alpha=0.5, color='orange', linewidth=7)
plt.plot(aromatic_feature_values, aromatic_trend_line_y, alpha=0.5, color='green', linewidth=7)

plt.xlabel(target_feature_name, fontsize=40, labelpad=25)
plt.ylabel('SHAP Value', fontsize=40, labelpad=25)

ax = plt.gca()
ax.spines['top'].set_linewidth(3)
ax.spines['right'].set_linewidth(3)
ax.spines['left'].set_linewidth(3)
ax.spines['bottom'].set_linewidth(3)
ax.tick_params(axis='both', which='major', width=3, length=10)

plt.gca().xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))

ax.yaxis.set_major_locator(MaxNLocator(nbins=6))

plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.legend(fontsize=40, frameon=False)
plt.show()



SHAP analysis baesd on OMPs classification into aromatic/aliphatic and high/low hydrophobic

In [ ]:
data_original = pd.read_excel(r'Original_data')

# Classification based on whether or not the number of aromatic rings is 0 and HBFrac is 0
aromatic_Hbond_indices = data_original[(data_original.iloc[:, 1] != 0) & ((data_original.iloc[:, 3] != 0))].index
aromatic_no_Hbond_indices = data_original[(data_original.iloc[:, 1] != 0) & ((data_original.iloc[:, 3] == 0))].index
non_aromatic_Hbond_indices = data_original[(data_original.iloc[:, 1] == 0) & ((data_original.iloc[:, 3] != 0))].index
non_aromatic_no_Hbond_indices = data_original[(data_original.iloc[:, 1] == 0) & ((data_original.iloc[:, 3] == 0))].index

# Local SHAP
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

shap_aromatic_Hbond_df = pd.DataFrame(columns=['Isotherm', *feature_names])
shap_aromatic_no_Hbond_df = pd.DataFrame(columns=['Isotherm', *feature_names])
shap_non_aromatic_Hbond_df = pd.DataFrame(columns=['Isotherm', *feature_names])
shap_non_aromatic_no_Hbond_df = pd.DataFrame(columns=['Isotherm', *feature_names])

for isotherm in sorted(data['Isotherm'].unique()):
    iso_indices = data[data['Isotherm'] == isotherm].index
    shap_values_iso = shap_values[iso_indices]
    
    if any(idx in aromatic_Hbond_indices for idx in iso_indices):
        shap_df_iso = pd.DataFrame(shap_values_iso, columns=feature_names)
        shap_df_iso['Isotherm'] = isotherm
        shap_aromatic_Hbond_df = pd.concat([shap_aromatic_Hbond_df, shap_df_iso], ignore_index=True)
    if any(idx in aromatic_no_Hbond_indices for idx in iso_indices):
        shap_df_iso = pd.DataFrame(shap_values_iso, columns=feature_names)
        shap_df_iso['Isotherm'] = isotherm
        shap_aromatic_no_Hbond_df = pd.concat([shap_aromatic_no_Hbond_df, shap_df_iso], ignore_index=True)
    if any(idx in non_aromatic_Hbond_indices for idx in iso_indices):
        shap_df_iso = pd.DataFrame(shap_values_iso, columns=feature_names)
        shap_df_iso['Isotherm'] = isotherm
        shap_non_aromatic_Hbond_df = pd.concat([shap_non_aromatic_Hbond_df, shap_df_iso], ignore_index=True)
    elif any(idx in non_aromatic_no_Hbond_indices for idx in iso_indices):
        shap_df_iso = pd.DataFrame(shap_values_iso, columns=feature_names)
        shap_df_iso['Isotherm'] = isotherm
        shap_non_aromatic_no_Hbond_df = pd.concat([shap_non_aromatic_no_Hbond_df, shap_df_iso], ignore_index=True)

shap_aromatic_Hbond_df = shap_aromatic_Hbond_df.sort_values(by='Isotherm')
shap_aromatic_no_Hbond_df = shap_aromatic_no_Hbond_df.sort_values(by='Isotherm')
shap_non_aromatic_Hbond_df = shap_non_aromatic_Hbond_df.sort_values(by='Isotherm')
shap_non_aromatic_no_Hbond_df = shap_non_aromatic_no_Hbond_df.sort_values(by='Isotherm')


In [ ]:
target_feature_name = feature_names[0]
target_feature_index = feature_names.get_loc(target_feature_name)

# LogD extraction
aromatic_Hbond_feature_values = data_original.iloc[aromatic_Hbond_indices, target_feature_index + 2]
aromatic_no_Hbond_feature_values = data_original.iloc[aromatic_no_Hbond_indices, target_feature_index + 2]
non_aromatic_Hbond_feature_values = data_original.iloc[non_aromatic_Hbond_indices, target_feature_index + 2]
non_aromatic_no_Hbond_feature_values = data_original.iloc[non_aromatic_no_Hbond_indices, target_feature_index + 2]

# SHAP extraction
aromatic_Hbond_shap_values = shap_aromatic_Hbond_df.iloc[:, target_feature_index + 1]
aromatic_no_Hbond_shap_values = shap_aromatic_no_Hbond_df.iloc[:, target_feature_index + 1]
non_aromatic_Hbond_shap_values = shap_non_aromatic_Hbond_df.iloc[:, target_feature_index + 1]
non_aromatic_no_Hbond_shap_values = shap_non_aromatic_no_Hbond_df.iloc[:, target_feature_index + 1]

# Trendline
aromatic_Hbond_trend_line = np.polyfit(aromatic_Hbond_feature_values, aromatic_Hbond_shap_values, 1)
aromatic_Hbond_trend_line_y = np.polyval(aromatic_Hbond_trend_line, aromatic_Hbond_feature_values)
aromatic_Hbond_r2 = r2_score(aromatic_Hbond_shap_values, aromatic_Hbond_trend_line_y)

aromatic_no_Hbond_trend_line = np.polyfit(aromatic_no_Hbond_feature_values, aromatic_no_Hbond_shap_values, 1)
aromatic_no_Hbond_trend_line_y = np.polyval(aromatic_no_Hbond_trend_line, aromatic_no_Hbond_feature_values)
aromatic_no_Hbond_r2 = r2_score(aromatic_no_Hbond_shap_values, aromatic_no_Hbond_trend_line_y)

non_aromatic_Hbond_trend_line = np.polyfit(non_aromatic_Hbond_feature_values, non_aromatic_Hbond_shap_values, 1)
non_aromatic_Hbond_trend_line_y = np.polyval(non_aromatic_Hbond_trend_line, non_aromatic_Hbond_feature_values)
non_aromatic_Hbond_r2 = r2_score(non_aromatic_Hbond_shap_values, non_aromatic_Hbond_trend_line_y)

non_aromatic_no_Hbond_trend_line = np.polyfit(non_aromatic_no_Hbond_feature_values, non_aromatic_no_Hbond_shap_values, 1)
non_aromatic_no_Hbond_trend_line_y = np.polyval(non_aromatic_no_Hbond_trend_line, non_aromatic_no_Hbond_feature_values)
non_aromatic_no_Hbond_r2 = r2_score(non_aromatic_no_Hbond_shap_values, non_aromatic_no_Hbond_trend_line_y)

aromatic_Hbond_slope = aromatic_Hbond_trend_line[0]
aromatic_no_Hbond_slope = aromatic_no_Hbond_trend_line[0]
non_aromatic_Hbond_slope = non_aromatic_Hbond_trend_line[0]
non_aromatic_no_Hbond_slope = non_aromatic_no_Hbond_trend_line[0]

# Local SHAP plotting
plt.rcParams['font.family'] = 'Arial'

plt.figure(figsize=(20, 8))
plt.plot(aromatic_Hbond_feature_values, aromatic_Hbond_trend_line_y, alpha=0.5, color='green', linewidth=7)
plt.plot(aromatic_no_Hbond_feature_values, aromatic_no_Hbond_trend_line_y, alpha=0.5, color='blue', linewidth=7)
plt.plot(non_aromatic_Hbond_feature_values, non_aromatic_Hbond_trend_line_y, alpha=0.5, color='orange', linewidth=7)
plt.plot(non_aromatic_no_Hbond_feature_values, non_aromatic_no_Hbond_trend_line_y, alpha=0.5, color='red', linewidth=7)

plt.plot(aromatic_Hbond_feature_values, aromatic_Hbond_shap_values, 'o', alpha=0.25, 
         color='green', label=f'Aromatic with H-bond (Slope={aromatic_Hbond_slope:.2f})', markersize=20)

plt.plot(aromatic_no_Hbond_feature_values, aromatic_no_Hbond_shap_values, 'o', alpha=0.25, 
         color='blue', label=f'Aromatic without H-bond (Slope={aromatic_no_Hbond_slope:.2f})', markersize=20)

plt.plot(non_aromatic_Hbond_feature_values, non_aromatic_Hbond_shap_values, 'o', alpha=0.25, 
         color='orange', label=f'Aliphatic with H-bond (Slope={non_aromatic_Hbond_slope:.2f})', markersize=20)

plt.plot(non_aromatic_no_Hbond_feature_values, non_aromatic_no_Hbond_shap_values, 'o', alpha=0.25, 
         color='red', label=f'Aliphatic without H-bond (Slope={non_aromatic_no_Hbond_slope:.2f})', markersize=20)


plt.xlabel(target_feature_name, fontsize=40, labelpad=25)
plt.ylabel('SHAP Value', fontsize=40, labelpad=25)

ax = plt.gca()
ax.spines['top'].set_linewidth(3)
ax.spines['right'].set_linewidth(3)
ax.spines['left'].set_linewidth(3)
ax.spines['bottom'].set_linewidth(3)

ax.tick_params(axis='both', which='major', width=3, length=10)

plt.gca().xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))

ax.yaxis.set_major_locator(MaxNLocator(nbins=6))

plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.legend(fontsize=35, frameon=False)

plt.show()
